Antes de comenzar, tendremos que para utilizar estas funciones, se necesita la función objetivo $f$, su gradiente $df$, y el hessiano $ddf$. Definiendo la estructura genérica para los métodos de descenso:

* `f`: Función objetivo
* `df`: Gradiente de la función objetivo.
* `ddf`: El hessiano de la función objetivo.
* `x0`: Punto de inicio.
* `a`: Step size.
* `maxIter`: Número máximo de pasos o iteraciones.
* `e`: Tolerancia para el criterio de paro.
* `stopCriterion`: Criterio de paro.

In [127]:
using LinearAlgebra

1. **Descenso Gradiente Naive con Dirección Aleatoria**

In [128]:
function GradNaiveRand(f, df, x0, a, maxIter, e, stopCriterion)
    # Definir el punto inicial
    x = x0

    # x_k es la secuencia de puntos x obtenidos en cada iteración
    x_k = [x0]

    # f_k es la secuencia de valores de la función objetivo en cada iteración
    f_k = [f(x0)]
    errors = []

    for k in 1:maxIter
        # Generar la dirección de descenso al gradiente de forma aleatoria
        d = randn(length(x))

        # Normalizar la dirección de descenso
        d = d/norm(d)

        # Calcular el nuevo punto
        x_new = x + a*d

        # Guardar el nuevo punto y el valor de la función objetivo
        push!(x_k, x_new)
        push!(f_k, f(x_new))

        # Calcular el error
        error = norm(df(x_new))

        # Guardar el error
        push!(errors, error)

        # Si se cumple el criterio de paro, terminar
        if error < e
            return x_new, x_k, f_k, errors, k, true
        end

        # Actualizar el punto actual
        x = x_new    
    end

    # Retornar el punto actual, la secuencia de puntos, la secuencia de valores de la función objetivo, la secuencia de errores, el número de iteraciones
    # Y un indicador que no converge
    return x, x_k, f_k, errors, maxIter, false
end

GradNaiveRand (generic function with 1 method)

2. **Descenso Máximo Naive**

In [129]:
function GradNaiveMax(f, df, x0, a, maxIter, e, stopCriterion)
    # Definir el punto inicial
    x = x0

    # x_k es la secuencia de puntos x obtenidos en cada iteración
    x_k = [x0]

    # f_k es la secuencia de valores de la función objetivo en cada iteración
    f_k = [f(x0)]

    errors = []

    for k in 1:maxIter
        # Calcular la dirección de descenso gradiente
        d = df(x)

        # Asegurarse de que x y d tengan las mismas dimensiones
        if length(x) != length(d)
            throw(DimensionMismatch("x y d deben tener las mismas dimensiones"))
        end

        # Calcular el nuevo punto
        x_new = x - a*d  # Cambiado a restar el gradiente

        # Guardar el nuevo punto y el valor de la función objetivo
        push!(x_k, x_new)
        push!(f_k, f(x_new))

        # Calcular el error
        error = norm(df(x_new))  # Cambiado a x_new

        # Guardar el error
        push!(errors, error)

        # Si se cumple el criterio de paro, terminar
        if error < e
            return x_new, x_k, f_k, errors, k, true
        end

        # Actualizar el punto actual
        x = x_new
    end

    # Retornar el punto actual, la secuencia de puntos, la secuencia de valores de la función objetivo, la secuencia de errores, el número de iteraciones
    # Y un indicador que no converge
    return x, x_k, f_k, errors, maxIter, false
end

GradNaiveMax (generic function with 1 method)

3. **Descenso Gradiente de Newton**

In [130]:
function GradNewton(f, df, x0, a, maxIter, e, stopCriterion)
    # Definir el punto inicial
    x = x0

    # x_k es la secuencia de puntos x obtenidos en cada iteración
    x_k = [x0]

    # f_k es la secuencia de valores de la función objetivo en cada iteración
    f_k = [f(x0)]

    errors = []

    # Inicializar la matriz identidad para el Hessiano
    H_I = I(length(x0))

    for k in 1:maxIter
        # Calcular la dirección de descenso
        d = - H_I * df(x)

        # Calcular el nuevo punto
        x_new = x + a*d

        # Guardar el nuevo punto y el valor de la función objetivo
        push!(x_k, x_new)
        push!(f_k, f(x_new))

        # Calcular el error
        error = norm(df(x))
        push!(errors, error)

        # Si se cumple el criterio de paro, terminar
        if error < e
            return x_new, x_k, f_k, errors, k, true
        end

        # Actualización del Hessiano Inverso
        s = x_new - x
        y = df(x_new) - df(x)
        rho = 1/(y'*s)
        H_I = (I -rho*s*y')*H_I*(I-rho*y*s') + rho*s*s'

        # Actualizar el punto actual
        x = x_new

    end

    return x, x_k, f_k, errors, maxIter, false
end

GradNewton (generic function with 1 method)

4. **Descenso Gradiente de Newton (Hessiano Exacto)**

In [131]:
function GradNewtonExact(f, df, ddf, x0, a, maxIter, e, stopCriterion)

    λ = 1e-6  # Regularización

    # Definir el punto inicial
    x = x0

    # x_k es la secuencia de puntos x obtenidos en cada iteración
    x_k = [x0]

    # f_k es la secuencia de valores de la función objetivo en cada iteración
    f_k = [f(x0)]

    errors = []

    for k in 1:maxIter
        # Obtener la dirección de descenso
        g = df(x)
        H = ddf(x) + λ * I

        # Resolver el sistema de ecuaciones
        d = -H\g

        # Calcular el nuevo punto
        x_new = x + a*d

        # Guardar el nuevo punto y el valor de la función objetivo
        push!(x_k, x_new)
        push!(f_k, f(x_new))

        # Calcular el error
        error = norm(df(x))
        push!(errors, error)

        # Si se cumple el criterio de paro, terminar
        if error < e
            return x_new, x_k, f_k, errors, k, true
        end

        # Actualizar el punto actual
        x = x_new
    end

    return x, x_k, f_k, errors, maxIter, false
end

GradNewtonExact (generic function with 1 method)

**Testeando la función $ f : \R^{10} \to \R $ dada por**

$$f(x) = \sum_{i=1}^{9} [100(x_{i+1} - x_{i}^{2})^{2} + (1-x_i)^2]$$

In [132]:
function f(x)
    n = length(x)
    fu = 0.0
    for i in 1:n-1
        fu += 100 * (x[i+1] - x[i]^2)^2 + (1 - x[i])^2
    end
    return fu
end

f (generic function with 1 method)

**Gradiente de la función**:

El gradiente $\nabla f(x_i)$

$$ \frac{\partial f}{\partial x_i} =
\begin{cases}
-400x_i(x_{i+1} - x_{i}^{2})-2(1-x_i), & \text{ si} \leq i \lt n\\
200(x_i - x_{i-1}^{2}), & \text{ si} \ \ i = n
\end{cases}
$$

In [133]:
function df(x)
    n = length(x)
    grad = zeros(n)
    for i in 1:n-1
        grad[i] = -400 * x[i] * (x[i+1] - x[i]^2) - 2 * (1 - x[i])
    end
    grad[n] = 200 * (x[n] - x[n-1]^2)
    return grad
end

df (generic function with 1 method)

**Hessiano de la función**:

El hessiano $H(f(x_1,x_2))$

$$
\frac{\partial ^2 f}{\partial x_{i}^{2}} = 1200x_{i}^{2} - 400x_{i+1} + 2, \ \ \ \ \ 1 \leq i \lt n \\

\frac{\partial ^2 f}{\partial x_{i} \partial x_{i+1}} = -400x_i, \ \ \ \ \ \text{ y su simétrico}
$$

In [134]:
function ddf(x)
    n = length(x)
    H = zeros(n, n)
    for i in 1:n-1
        H[i,i] = 1200 * x[i]^2 - 400 * x[i+1] + 2
        H[i,i+1] = -400 * x[i]
        H[i+1,i] = H[i,i+1]  # Simétrico
    end
    H[n,n] = 200
    return H
end

ddf (generic function with 1 method)

**Punto inicial**: $x_0 = (-1.2, 1, 1, . . . , 1, -1.2, 1)^{T}$


In [135]:
x0 = [-1.2; ones(8); -1.2; 1]

11-element Vector{Float64}:
 -1.2
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
 -1.2
  1.0

In [136]:
α = 0.001  # Tamaño de paso
maxIter = 10000  # Máximo de iteraciones
ε = 1e-6  # Tolerancia
stop_criterion = :gradiente  # Criterio de paro

:gradiente

### Algoritmo de Descenso Gradiente Naive con Dirección Aleatoria

In [148]:
α = 0.001  # Tamaño de paso
maxIter = 10000
x_opt, xk, fk, errores, iteraciones, convergencia = GradNaiveRand(f, df, x0, α, maxIter, ε, stop_criterion)
println("Solución óptima: ", x_opt)
println("Número de iteraciones: ", iteraciones)
println("Valor de f en la solución: ", f(x_opt))
println("Convergencia: ", convergencia)

Solución óptima: [-1.2042246762984767, 1.053117798955782, 1.0353653774198246, 0.9712467115148897, 1.0222928294449913, 0.9905055202231092, 1.0101394276023004, 1.013853609184866, 0.9988808303588836, -1.1990077649564344, 0.9352705427169383]
Número de iteraciones: 10000
Valor de f en la solución: 535.9317627056361
Convergencia: false


### Algoritmo de Descenso Máximo Naive

In [152]:
α = 0.0001
maxIter = 1000000
x_opt, xk, fk, errores, iteraciones, convergencia = GradNaiveMax(f, df, x0, α, maxIter, ε, stop_criterion)
graficar_error_algoritmo(errores, "Gradiente Aleatorio")
println("Solución óptima: ", x_opt)
println("Número de iteraciones: ", iteraciones)
println("Valor de f en la solución: ", f(x_opt))
println("Convergencia: ", convergencia)

plot (generic function with 4 methods)

Solución óptima: [-0.9949747447323938, 0.999999995837412, 0.9999999916581671, 0.9999999832829538, 0.9999999664990131, 0.9999999328639697, 0.9999998654592923, 0.9999997303802246, 0.9999994596816122, 0.9999989172013792, 0.9999978300710032]
Número de iteraciones: 321671
Valor de f en la solución: 3.9899748022582036
Convergencia: true


### Algoritmo de Descenso Gradiente de Newton

In [156]:
α = 0.00001
maxIter = 4000000
x_opt, xk, fk, errores, iteraciones, convergencia = GradNewton(f, df, x0, α, maxIter, ε, stop_criterion)
println("Solución óptima: ", x_opt)
println("Número de iteraciones: ", iteraciones)
println("Valor de f en la solución: ", f(x_opt))
println("Convergencia: ", convergencia)

Solución óptima: [-0.9949747458908894, 0.9999999975662875, 0.9999999951197805, 0.9999999902153807, 0.9999999803781668, 0.9999999606493017, 0.9999999210747166, 0.9999998417044168, 0.9999996825673902, 0.9999993615804497, 0.9999987204945936]
Número de iteraciones: 2341126
Valor de f en la solución: 3.989974805723667
Convergencia: true


### Algoritmo de Descenso Gradiente de Newton (Hessiano Exacto)

In [158]:
maxIter = 6000000
x_opt, xk, fk, errores, iteraciones, convergencia = GradNewtonExact(f, df, ddf, x0, α, maxIter, ε, stop_criterion)
println("Solución óptima: ", x_opt)
println("Número de iteraciones: ", iteraciones)
println("Valor de f en la solución: ", f(x_opt))
println("Convergencia: ", convergencia)

Solución óptima: [0.07664243495304889, -0.054363947975469486, 0.09989884519040765, -0.03508220253069501, 0.14865467553206307, -0.006542642684043277, 0.7668635501806548, 0.5865591688254453, 0.3494405293481995, 0.13074551445160126, 0.01709438894321838]
Número de iteraciones: 6000000
Valor de f en la solución: 69.56101700777683
Convergencia: false


**Óptimo**: $x^{*} = (1, 1, ... , 1)^{T}, \ \ \ f(x^{*}) = 0$